<a href="https://colab.research.google.com/github/amittkatz/Information-Retrieval-/blob/main/Information_Retrieval_ALMOGandAMIT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Information Retrieval Task



In [35]:
import requests #for making HTTP requests in Python
from bs4 import BeautifulSoup # pulling data from HTML or XML files

In [36]:
query = "Alzheimer signs"


In [37]:
r = requests.get('https://www.google.com/search?q={}'.format(query))

In [38]:
soup = BeautifulSoup(r.text, "html.parser")

In [39]:
links = []
for item in soup.find_all('a'):
    links.append(item.get('href'))

In [40]:
final = []
for item in links:
    aaa = item[0:7]
    if aaa =="/url?q=":
        final.append(item)

saving the links as s doc

In [41]:
for i in range(1,11):
  webpage = requests.get('https://www.google.com/' + final[i]) #final[9] refers to the first web page link
  webpagetext = BeautifulSoup(webpage.text, "html.parser")
  all_p = webpagetext.find_all('p')
  text = ""
  for item in all_p:
      text = text + "\n" + item.get_text()
      with open("doc_"+ str(i)+".txt", "w") as doc_link:
        doc_link.write(text)

Implementing Information Retrieval using Boolean Query

In [42]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
nltk.download('punkt')
Stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq

def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
    # return freq
        
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

    

Simplifying the Information

In [44]:
all_words = []
dict_global = {}
file_folder = '*.txt'
idx = 1
files_with_index = {}
for f in glob.glob(file_folder):
    print(f)
    fname = f
    f = open(f , "r")
    text = f.read()
    text = remove_special_characters(text)
    # Create text that ready for regex work
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words.insert(0, fname)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    words = [PorterStemmer().stem(word) for word in words]

    text = ','.join(words) + "\n"

    with open("word_docs.txt", "a") as docs:
      # print(words)
      docs.write(text)
   
unique_words_all = set(dict_global.keys())

doc_6.txt
doc_1.txt
doc_4.txt
doc_9.txt
doc_10.txt
doc_2.txt
doc_8.txt
doc_7.txt
doc_5.txt
doc_3.txt


changing the doc to csv

In [45]:
os.rename('word_docs.txt','word_docs.csv')

In [46]:
import pandas
# module to read the contents of the file from a csv file
 
from contextlib import redirect_stdout
# module to redirect the output to a text file
 
terms = []
# list to store the terms present in the documents
 
keys = []
# list to store the names of the documents
 
vec_Dic = {}
# dictionary to store the name of the document and the boolean vector as list
 
dicti = {}
# dictionary to store the name of the document and the terms present in it as a
# vector
 
dummy_List = []
# list for performing some operations and clearing them
 
 
def filter(documents, rows, cols):
    '''function to read and separate the name of the documents and the terms
    present in it to a separate list  from the data frame and also create a
    dictionary which has the name of the document as key and the terms present in
    it as the list of strings  which is the value of the key'''
 
    for i in range(rows):
        for j in range(cols):
            # traversal through the data frame
 
            if(j == 0):
                # first column has the name of the document in the csv file
                keys.append(documents.loc[i].iat[j])
               
            else:
                dummy_List.append(documents.loc[i].iat[j])
                # dummy list to update the terms in the dictionary
 
                if documents.loc[i].iat[j] not in terms:
                    # add the terms to the list if it is not present else continue
                    terms.append(documents.loc[i].iat[j])
 
        copy = dummy_List.copy()
        # copying the dummy list to a different list
 
        dicti.update({documents.loc[i].iat[0]: copy})
        # adding the key value pair to a dictionary
 
        dummy_List.clear()
        # clearing the dummy list
 
 
def bool_Representation(dicti, rows, cols):
    '''In this function we get a boolean representation of the terms present in the
    documents in the form of lists, later we create a dictionary which contains
    the name of the documents as key and value as the list of boolean values
    representing the terms present in the document'''
 
    # we sort the elements in the alphabetical order for the convience, the order
    # of the term does not make any difference
 
    for i in (dicti):
        # for every document in the dictionary we check for each string present in
        # the list
 
        for j in terms:
            # if the string is present in the list we append 1 else we append 0
 
            if j in dicti[i]:
                dummy_List.append(1)
            else:
                dummy_List.append(0)
            # appending 1 or 0 for obtaining the boolean representation
 
        copy = dummy_List.copy()
        # copying the dummy list to a different list
 
        vec_Dic.update({i: copy})
        # adding the key value pair to a dictionary
 
        dummy_List.clear()
        # clearing the dummy list
 
 
def query_Vector(query):
    '''In this function we represent the query in the form of boolean vector'''
 
    qvect = []
    # query vector which is returned at the end of the function
 
    for i in terms:
        # if the word present in the list of terms is also present in the query
        # then append 1 else append 0
 
        if i in query:
            qvect.append(1)
        else:
            qvect.append(0)
 
    return qvect
    # return the query vector which is obtained in the boolean form
 
 
def prediction(q_Vect):
    '''In this function we make the prediction regarding which document is related
    to the given query by performing the boolean operations'''
 
    dictionary = {}
    listi = []
    count = 0
    # initialisation of the dictionary , list and a variable which is further
    # required for performing the computation
 
    term_Len = len(terms)
    # number of terms present in the term list
 
    for i in vec_Dic:
        # for every document in the dictionary containing the terms present in it
        # the form of boolean vector
 
        for t in range(term_Len):
            if(q_Vect[t] == vec_Dic[i][t]):
                # if the words present in the query is also present in the
                # document or if the words present in the query is also absent in
                # the document
 
                count += 1
                # increase the value of count variable by one
                # the condition in which words present in document and absent in
                #query , present in query and absent in document is not considered
 
        dictionary.update({i: count})
        # dictionary updation here the name of the document is the key and the
        # count variable computed earlier is the value
 
        count = 0
        # reinitialisaion of count variable to 0
 
    for i in dictionary:
        listi.append(dictionary[i])
        # here we append the count value to list
 
    listi = sorted(listi, reverse=True)
    # we sort the list in the descending order which is needed to rank the
    #documents according to the relevance
 
    ans = ' '
    # variable to store the name of the document which is most relevant
 
    with open('output.txt', 'w') as f:
        with redirect_stdout(f):
            # to redirect the output to a text file
 
            print("ranking of the documents")
 
            for count, i in enumerate(listi):
                key = check(dictionary, i)
                # Function call to get the key when the value is known
                if count == 0:
                    ans = key
                    # to store the name of the document which is most relevant
 
                print(key, "rank is", count+1)
                # print the name of the document along with its rank
 
                dictionary.pop(key)
                # remove the key from the dictionary after printing
 
            print(ans, "is the most relevant document for the given query")
            # to print the name of the document which is most relevant
 
 
def check(dictionary, val):
    '''Function to return the key when the value is known'''
 
    for key, value in dictionary.items():
        if(val == value):
            # if the given value is same as the value present in the dictionary
            # return the key
 
            return key
 
 


In [47]:
def main():
    documents = pandas.read_csv(r'word_docs.csv', on_bad_lines='skip')
    # to read the data from the csv file as a dataframe
 
    
    rows = len(documents)
    # to get the number of rows
   
    cols = len(documents.columns)
    # to get the number of columns

    
    filter(documents, rows, cols)
    # function call to read and separate the name of the documents and the terms
    # present in it to a separate list  from the data frame and also create a
    # dictionary which has the name of the document as key and the terms present in
    # it as the list of strings  which is the value of the key
 
    bool_Representation(dicti, rows, cols)
    # In this function we get a boolean representation of the terms present in the
    # documents in the form of lists, later we create a dictionary which contains
    # the name of the documents as key and value as the list of boolean values
    #representing the terms present in the document
    print(vec_Dic)
 
    print("Enter query")
    query = input()
    # to get the query input from the user, the below input is given for obtaining
    # the output as in output.txt file
    # hockey is a national sport
 
    query = query.split(' ')
    # spliting the query as a list of strings 
    
    q_Vect = query_Vector(query)
    # function call to represent the query in the form of boolean vector
    
    prediction(q_Vect)
    print(q_Vect)
    print(prediction(q_Vect))
    # Function call to make the prediction regarding which document is related to
    # the given query by performing the boolean operations
 
 
main()

{'doc_1.txt': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Boolean matrix

In [48]:
import pandas as pd
b_matrix = np.array([vec_Dic[i] for i in list(vec_Dic.keys())])
df = pd.DataFrame(b_matrix, index=list(vec_Dic.keys()))
df.T

,doc_1.txt,doc_4.txt,doc_9.txt,doc_10.txt,doc_2.txt,doc_8.txt,doc_7.txt,doc_5.txt,doc_3.txt
0,1,1,1,1,1,1,1,1,1
1,1,1,0,0,1,0,0,1,1
2,1,1,0,0,1,0,0,1,1
3,1,1,0,1,1,0,0,1,1
4,1,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
1231,0,0,0,0,0,0,0,0,0
1232,0,0,0,0,0,0,0,0,0
1233,0,0,0,0,0,0,0,0,0
1234,0,0,0,0,0,0,0,0,0
